## Manage Database Credentials using GCP Secret Manager

Here is the process involved to get secret details as part of the applications.
* Create Secret Manager Client Object
* Get Secret Details
* Use Secret Details (to connect to Databases)

In [2]:
from google.cloud import secretmanager

In [3]:
client = secretmanager.SecretManagerServiceClient()

In [ ]:
client.access_secret_version?

In [4]:
'''
project_id = 'tidy-fort-361710'
secret_id = 'retailsecret'
version_id = 1
'''
project_id = "udemydataengineering"
secret_id = "udemy-retail-gcpsecret"
version_id = 1

In [5]:
secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'

In [6]:
secret_name

'projects/udemydataengineering/secrets/udemy-retail-gcpsecret/versions/1'

In [8]:
response = client.access_secret_version(name=secret_name)

In [9]:
type(response)

google.cloud.secretmanager_v1.types.service.AccessSecretVersionResponse

In [10]:
response.payload

data: "{\"host\":\"35.232.183.213\", \"database\":\"udemy_retail_db\", \"user\":\"udemy_retail_user\", \"password\": \"udemy-retail-dbpassword\"}"
data_crc32c: 2493074567

In [11]:
response.payload.data

b'{"host":"35.232.183.213", "database":"udemy_retail_db", "user":"udemy_retail_user", "password": "udemy-retail-dbpassword"}'

In [12]:
response.payload.data.decode('utf-8')

'{"host":"35.232.183.213", "database":"udemy_retail_db", "user":"udemy_retail_user", "password": "udemy-retail-dbpassword"}'

In [13]:
import json

In [14]:
secret_details = json.loads(response.payload.data.decode('utf-8'))

In [ ]:
secret_details.keys()

In [ ]:
secret_details['user']

In [ ]:
secret_details['password']

In [ ]:
import json
from google.cloud import secretmanager

def get_secret_details(secret_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(name=secret_name)
    return json.loads(response.payload.data.decode('utf-8'))

In [ ]:
project_id = 'tidy-fort-361710'
secret_id = 'retailsecret'
version_id = 1

secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'
secret_details = get_secret_details(secret_name)

In [ ]:
secret_details

In [ ]:
conn_uri = 'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
conn_uri.format(port=5432, **secret_details)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri.format(port=5432, **secret_details)
)

In [ ]:
df